In [30]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from Preprocessing.to_embedding import WordEmbedding
from Preprocessing.data_format import formatting
from Preprocessing.helper_functions import import_embedding, embedding_matrix_word2vec
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence

In [31]:
data = pd.read_csv('phase2_baby_all.csv')

data["sentence"] = data["sentence"].astype(str)
data["sentence"] = data["sentence"].apply(text_to_word_sequence)

embedding_size = 300 #number of feature weights in embeddings
max_len = 400

In [32]:
embedding = WordEmbedding(num_features = embedding_size)
WordEmbedding.fit(embedding, data["sentence"])
WordEmbedding.size(embedding)

Total number of words in the vocabulary:  (2979, 300)


In [33]:
#Save word embedding to dataframe
#train_embeddings = WordEmbedding.to_pd(embedding, X_train)

#Save Save embeddings to file
WordEmbedding.to_file(embedding)

In [34]:
embeddings_index = import_embedding('trained_embedding_word2vec.txt')

## 2. Vectorize text data

In [35]:
def vectorize(data, tokenizer ,max_len):
    sequences = tokenizer.texts_to_sequences(data)
    padding = pad_sequences(sequences, maxlen = max_len)
    
    return padding

### Check train/dev sets

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional, GlobalMaxPool1D, Dropout
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from sklearn.model_selection import StratifiedKFold


cvscores = []
for group_id in range(1,9):
    train = data.loc[data['group_id'] != group_id]
    X_train = train['sentence']
    y_train = train['sentiment']
    
    test = data.loc[data['group_id'] == group_id]
    X_test = test['sentence']
    y_test = test['sentiment']
    
    y_train = pd.get_dummies(y_train)
    y_test = pd.get_dummies(y_test)
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    embedding_matrix, num_words = embedding_matrix_word2vec(word_index, embedding_size, embeddings_index)
    
    X_train = vectorize(X_train, tokenizer , max_len)
    X_test = vectorize(X_test, tokenizer, max_len)

    # Define Model
    model = Sequential()
    model.add(Embedding(num_words, 
                        embedding_size,
                        input_length = max_len,
                         dropout=0.2))
    model.add(Bidirectional(LSTM(128, return_sequences = True)))
    model.add(GlobalMaxPool1D())
    model.add(Dense(20, activation="relu"))
    model.add(Dropout(0.05))
    model.add(Dense(3, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train, y_train, batch_size = 256, epochs = 4, verbose = 0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

/Users/davidmortensen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.


KeyboardInterrupt: 